<a href="https://colab.research.google.com/github/AnudeepReddy-Katta/nlp-journey/blob/main/DataPreparation/Preparing_the_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext.legacy import data 
# The torchtext package consists of data processing utilities and popular datasets for natural language.

In [2]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic  = True 
# To have reproducability internally

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm') # Ability to define a preprocessing pipeline

LABEL = data.LabelField(dtype=torch.float)

In [3]:
from torchtext.legacy import datasets
# The raw text iterators for common NLP datasets

In [4]:
train_data, test_data = datasets.IMDB.splits(TEXT,LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 28.9MB/s]


In [5]:
print(len(train_data))
print(len(test_data))

25000
25000


In [6]:
print(vars(train_data.examples[0]))

{'text': ['This', 'film', 'is', 'about', 'a', 'man', 'who', 'has', 'been', 'too', 'caught', 'up', 'with', 'the', 'accepted', 'convention', 'of', 'success', ',', 'trying', 'to', 'be', 'ever', 'upwardly', 'mobile', ',', 'working', 'hard', 'so', 'that', 'he', 'could', 'be', 'proud', 'of', 'owning', 'his', 'own', 'home', '.', 'He', 'assumes', 'this', 'is', 'all', 'there', 'is', 'to', 'life', 'until', 'he', 'accidentally', 'takes', 'up', 'dancing', ',', 'all', 'because', 'he', 'wanted', 'to', 'get', 'a', 'closer', 'look', 'of', 'a', 'beautiful', 'girl', 'that', 'he', 'sees', 'by', 'the', 'dance', 'studio', 'everyday', 'while', 'riding', 'the', 'subway', 'on', 'his', 'way', 'home.<br', '/><br', '/>His', 'was', 'infatuated', 'with', 'her', 'at', 'first', ',', 'going', 'to', 'the', 'dance', 'class', 'just', 'to', 'idolize', 'her', ',', 'but', 'he', 'eventually', 'lets', 'himself', 'go', 'and', 'gets', 'himself', 'into', 'the', 'dancing', '.', 'It', 'eventually', 'becomes', 'apparent', 'to', 'h

In [7]:
# Train-valid data split

import random 

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

17500
7500
25000


In [9]:
# Building vocab

MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [10]:
print(len(TEXT.vocab))
print(len(LABEL.vocab))

2

In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202412), (',', 192759), ('.', 165751), ('and', 109636), ('a', 109396), ('of', 100712), ('to', 93894), ('is', 76323), ('in', 60993), ('I', 54132), ('it', 53828), ('that', 48896), ('"', 44196), ("'s", 43234), ('this', 42358), ('-', 36941), ('/><br', 35289), ('was', 34903), ('as', 30155), ('with', 29886)]


In [13]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [15]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [16]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    # iterator that batches examples of similar lengths together
    (train_data,valid_data,test_data),
    batch_size = BATCH_SIZE,
    device = device
)